In [1]:
!pip install TTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.7/736.7 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.1/581.1 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 38.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [2]:
import os

In [3]:
# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

BaseDatasetConfig provides a configuration template for managing dataset-related settings in text-to-speech (TTS) applications

In [4]:
output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [5]:
!wget -O tts_train_dir/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 

--2023-05-18 04:59:30--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘tts_train_dir/LJSpeech-1.1.tar.bz2’

tts_train_dir/LJSpe 100%[===================>]   2.56G  13.5MB/s    in 3m 17s  

2023-05-18 05:02:47 (13.3 MB/s) - ‘tts_train_dir/LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [6]:
!tar -xf tts_train_dir/LJSpeech-1.1.tar.bz2

In [7]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path="/content/LJSpeech-1.1")

In [8]:
from TTS.tts.configs.tacotron_config import TacotronConfig

config = TacotronConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,  #This value means that testing will be performed immediately without any delay after each epoch of training. The value -1 indicates that there is no specific number of epochs to wait before starting the testing phase.
    epochs=30,
    text_cleaner="phoneme_cleaners", 
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)


Note: A text cleaner is responsible for performing various text normalization operations to convert the raw input text into a cleaner representation suitable for TTS processing. The "phoneme_cleaners" text cleaner is likely designed to convert the input text into phonemes, which are the smallest units of sound in a language.

By using the "phoneme_cleaners" text cleaner, the TTS model will work with phoneme sequences instead of raw text, enabling it to generate speech that corresponds to the phonetic representation of the input text.

Now, we will initialize the audio processor using AudioProcessor class, responsible for handling feature extraction from audio and performing audio I/O operations.

In [9]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


Next we will initialize the tokenizer which is used to convert text to sequences of token IDs. If characters are not defined in the config, default characters are passed to the config.

In [10]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of [text, audio_file_path, speaker_name].

In [11]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 13100 files in /content/LJSpeech-1.1


In [12]:
from TTS.tts.models.tacotron import Tacotron
model = Tacotron(config, ap, tokenizer, speaker_manager=None)

In [13]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 12
 | > Num. of Torch Threads: 6
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir/run-May-18-2023_05+07AM-0000000

 > Model has 6803235 parameters


In [ ]:
trainer.fit()


 > EPOCH: 0/30
 --> tts_train_dir/run-May-18-2023_05+07AM-0000000


[*] Pre-computing phonemes...


  0%|          | 4/12969 [00:00<31:21,  6.89it/s]

ɪnstɛd əv weɪtɪŋ ðɛɹ, ɔzwɔld əpɛɹəntli wɛnt æz fɑɹ əweɪ æz hi kʊd ænd bɔɹdɪd ðə fɚst oʊk klɪf bʌs wɪt͡ʃ keɪm əlɔŋ
 [!] Character '͡' not found in the vocabulary. Discarding it.


 16%|█▌        | 2059/12969 [01:13<05:24, 33.64it/s]

...


   --> STEP: 7/406 -- GLOBAL_STEP: 11375
     | > decoder_loss: 1.39288  (1.39249)
     | > postnet_loss: 1.96537  (1.95619)
     | > stopnet_loss: 0.02312  (0.02609)
     | > ga_loss: 0.00931  (0.01172)
     | > decoder_diff_spec_loss: 0.20061  (0.20009)
     | > postnet_diff_spec_loss: 0.43613  (0.43959)
     | > decoder_ssim_loss: 0.78706  (0.78518)
     | > postnet_ssim_loss: 0.78015  (0.77160)
     | > loss: 1.46022  (1.47099)
     | > align_error: 0.95801  (0.95331)
     | > amp_scaler: 524288.00000  (524288.00000)
     | > grad_norm: 0.35726  (0.36777)
     | > current_lr: 0.00000 
     | > step_time: 0.48790  (0.51258)
     | > loader_time: 0.00650  (0.00706)


   --> STEP: 32/406 -- GLOBAL_STEP: 11400
     | > decoder_loss: 1.39003  (1.38755)
     | > postnet_loss: 1.91459  (1.94552)
     | > stopnet_loss: 0.01545  (0.01963)
     | > ga_loss: 0.00595  (0.00805)
     | > decoder_diff_spec_loss: 0.20058  (0.20360)
     | > postnet_diff_spec_loss: 0.43898  (0.44265)
     | > dec



> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 3 not found characters:
	| > ͡
	| > “
	| > ”
| > Number of instances : 131
 | > Preprocessing samples
 | > Max text length: 174
 | > Min text length: 20
 | > Avg text length: 100.76335877862596
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 34739.0
 | > Avg audio length: 144033.41221374046
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.01410 (-0.00121)
     | > avg_decoder_loss: 1.77443 (-0.03161)
     | > avg_postnet_loss: 1.99150 (-0.07450)
     | > avg_stopnet_loss: 0.04115 (-0.00036)
     | > avg_ga_loss: 0.00350 (-0.00000)
     | > avg_decoder_diff_spec_loss: 0.23732 (+0.00087)
     | > avg_postnet_diff_spec_loss: 0.44424 (-0.00283)
     | > avg_decoder_ssim_loss: 0.81485 (-0.00338)
     | > avg_postnet_ssim_loss: 0.56815 (-0.01597)
     | > avg_loss: 1.51629 (-0.03222)
     | > avg_align_error: 0.98900 (+0.00006)

 > BEST MODEL : tts_train_dir/run-May-18-2023_05+07AM-0000000/best_model_11774.pth

 > EPOCH: 29/30
 --> tts_train_dir/run-May-18-2023_05+07AM-0000000

 > TRAINING (2023-05-18 13:59:11) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 3 not found characters:
	| > ͡
	| > “
	| > ”
| > Number of instances : 12969
 | > Preprocessing samples
 | > Max text length: 188
 | > Min text length: 13
 | > Avg text length: 100.90014650319993
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 24499.0
 | > Avg audio length: 144984.29755570978
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 1/406 -- GLOBAL_STEP: 11775
     | > decoder_loss: 1.40777  (1.40777)
     | > postnet_loss: 1.91316  (1.91316)
     | > stopnet_loss: 0.03420  (0.03420)
     | > ga_loss: 0.01490  (0.01490)
     | > decoder_diff_spec_loss: 0.20689  (0.20689)
     | > postnet_diff_spec_loss: 0.44050  (0.44050)
     | > decoder_ssim_loss: 0.77325  (0.77325)
     | > postnet_ssim_loss: 0.73852  (0.73852)
     | > loss: 1.47871  (1.47871)
     | > align_error: 0.94379  (0.94379)
     | > amp_scaler: 524288.00000  (524288.00000)
     | > grad_norm: 0.39950  (0.39950)
     | > current_lr: 0.00000 
     | > step_time: 0.73280  (0.73283)
     | > loader_time: 0.00650  (0.00647)


   --> STEP: 26/406 -- GLOBAL_STEP: 11800
     | > decoder_loss: 1.36118  (1.36507)
     | > postnet_loss: 1.85448  (1.87202)
     | > stopnet_loss: 0.01589  (0.02081)
     | > ga_loss: 0.00635  (0.00857)
     | > decoder_diff_spec_loss: 0.20994  (0.20471)
     | > postnet_diff_spec_loss: 0.44372  (0.43892)
     | > dec

In [1]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-18 14:07:01.232174: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-18 14:07:01.288303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 14:07:02.182618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required b

In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
!tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

In [ ]:
import IPython
IPython.display.Audio("out.wav")